# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where T
    lattice=T(a) * I(3)  # lattice is a cube of ``a`` Bohrs
    He = ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))
    atoms = [He => [[1/2; 1/2; 1/2]]]  # Helium at the center of the box

    model = model_DFT(lattice, atoms, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...), tol=tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770746680680         NaN   2.96e-01   0.80    8.0
  2   -2.772136321851   -1.39e-03   5.08e-02   0.80    1.0
  3   -2.772170660288   -3.43e-05   1.66e-03   0.80    2.0
  4   -2.772170722112   -6.18e-08   9.93e-05   0.80    2.0
  5   -2.772170722997   -8.85e-10   2.47e-05   0.80    2.0
n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770680500504         NaN   2.96e-01   0.80    8.0
  2   -2.772039704777   -1.36e-03   5.09e-02   0.80    1.0
  3   -2.772083267663   -4.36e-05   1.43e-03   0.80    2.0
  4   -2.772083414817   -1.47e-07   2.30e-04   0.80    2.0
  5   -2.772083417697   -2.88e-09   5.36e-05   0.80    2.0


1.7733877820589181

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770758117984         NaN   3.00e-01   0.80    9.0
  2   -2.772058452768   -1.30e-03   4.70e-02   0.80    1.0
  3   -2.772082305222   -2.39e-05   4.79e-03   0.80    2.0
  4   -2.772083417350   -1.11e-06   1.01e-04   0.80    2.0
  5   -2.772083417800   -4.50e-10   1.38e-05   0.80    2.0

Polarizability via ForwardDiff:       1.772548348098508
Polarizability via finite difference: 1.7733877820589181
